In [188]:
from tqdm.notebook import tqdm

In [48]:
def read_input(path):
    with open(path) as file:
        digits = [int(d) for d in file.read().strip()]
        # files = digits[::2]
        # gaps = digits[1::2]
    return digits

In [96]:
test = read_input("test/test_09.txt")
test

[2, 3, 3, 3, 1, 3, 3, 1, 2, 1, 4, 1, 4, 1, 3, 1, 4, 0, 2]

In [49]:
input = read_input("input/input_09.txt")

In [225]:
def expand_disk_map(disk_map, file_ids=None):
    blocks = []
    if len(disk_map) % 2 == 1:
        disk_map = disk_map + [0]
    if file_ids is None:
        file_ids = iter(range(len(disk_map) // 2))
    else:
        file_ids = iter(file_ids)
    for i, digit in enumerate(disk_map):
        if i % 2 == 0: # digit is a file
            file_id = next(file_ids)
            blocks += [file_id] * digit # fill digit blocks with file_id
        else: # digit is blank space
            blocks += ["."] * digit
    return blocks

In [63]:
def backfill_disk_map(blocks):

    pbar = tqdm()
    while "." in blocks:
        first_gap = blocks.index(".")
        last_elem = blocks.pop()
        if isinstance(last_elem, int):
            blocks[first_gap] = last_elem
        pbar.update(1)
    pbar.close()
    return blocks


In [226]:
def q9a(disk_map):
    blocks = expand_disk_map(disk_map)
    backfilled = backfill_disk_map(blocks)
    checksum = sum([i * id for i, id in enumerate(backfilled)])
    return checksum

In [227]:
q9a(test)

0it [00:00, ?it/s]

1928

In [213]:
q9a(input)

0it [00:00, ?it/s]

6401092019345

In [242]:
def q9b(disk_map):
    files = disk_map[::2]
    gaps = disk_map[1::2] + [0] # gaps are all after the file of the same index. pad a 0 at end
    file_ids = list(range(len(files)))

    for file_index, file_size in tqdm(reversed(list(enumerate(files)))):
        file_origin = file_ids.index(file_index) # the actual current location of the file we are working on
        try: # gap must be bigger than file size. only check up to gaps before the file
            first_gap_index = next(gap_index for gap_index, gap_size in enumerate(gaps) if gap_size >= file_size and gap_index < file_origin)
        except StopIteration:
            # print(f"did not reorder for {file_index}")
            continue
        # print(f"reordering for {file_index}")
        gaps[file_origin - 1] = gaps[file_origin - 1] + file_size + gaps[file_origin] # old file location has gaps on either side combine
        del gaps[file_origin]
        gaps.insert(first_gap_index, 0) # file guaranteed to be moved next to existing file so add 0 gap
        gaps[first_gap_index + 1] -= file_size # gap to right of file reduced by file size
        del files[file_origin]
        files.insert(first_gap_index + 1, file_size)
        del file_ids[file_origin]
        file_ids.insert(first_gap_index + 1, file_index)
    
    new_disk_map = [val for pair in zip(files, gaps) for val in pair]
    blocks = expand_disk_map(new_disk_map, file_ids)

    checksum = 0
    for i, block in enumerate(blocks):
        if isinstance(block, str):
            continue
        else:
            checksum += i * block

    return checksum

In [243]:
q9b(test)

0it [00:00, ?it/s]

2858

In [244]:
q9b(input)

0it [00:00, ?it/s]

6431472344710